# 1.準備

- このファイル(`main.ipynb`)と同じディレクトリに`learner.py`をはじめとして複数個ある`py`ファイルを`python3`と`MeCab`の両方が利用可能な環境へダウンロードしましょう。

- Google Driveの`マイ ドライブ/Colab Notebooks`ディレクトリ直下に入れると、Colaboratryにてこのファイルを開くことが出来ます。  
この場合、次のセルを実行してください。

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/')

# 2. データ取得
`リスト化`関数を用いて、文章を文のリストへ変換します。  
コード中の`ここに文章をコピペしてください`から始まる文字列を、用意した文章に置き換えてください。  
また、`リスト化`関数の第2引数は、文のリストへ与える変数名です。

In [3]:
def リスト化(文章, 変数名):
    文章 = 文章.replace("\n", "").replace("\r", "")

    文のリスト = [文+"。" for 文 in 文章.split("。") if 文 != ""]

    print(変数名, " = [")
    for 文 in 文のリスト[:-1]:
        print('"' + 文 + '",')
    else:
        print('"' + 文のリスト[-1] + '"]')

リスト化(\
"""
ここに文章をコピペしてください。

すると、文章が句点ごとに区切られ、文のリストに変換されます。
""", "文のリスト")



文のリスト  = [
"ここに文章をコピペしてください。",
"すると、文章が句点ごとに区切られ、文のリストに変換されます。"]


# 3.学習
第2章で用意した文のリストたちから文を取り出し、1文ずつ`学習器`へ与えると、学習を開始します。  
次のコードの最後の行について、変数名を適切に置き換えて実行してみてください。  

また、`学習させる`関数は、次の方針に従って設計されています。  
1. 定期的に、特殊単語「EOS」1つからなる特殊な文に対する予測を行う。
この結果を「認知ベクトル」と呼ぶことにする。
2. 認知ベクトルの成分が最小になっているインデックスに対応するラベルに対応する文を重点的に学習する

この方針に従うことで、認知ベクトルは最終的に一様に近づく。これが「認知の歪み」のない状態である。



In [ ]:
m = 10
n = 2
lr= 0.01
# ---
from learner import 学習器 as 学習器型
from wordmatrix import 単語行列 as 単語行列型
import random

try:
    単語行列.__del__()
    del 単語行列
except NameError:
    pass
単語行列 = 単語行列型.get_instance(m, n, lr)

学習器 = 学習器型(m, n, 単語行列)

def 学習させる(学習器, データ, 回数):
    """
    学習器: 未学習の学習器型オブジェクト
    データ: ラベル番号から文のリストへのリスト
    もっとも不利なラベルを認知ベクトルによって測定し、そのラベルを指定回数だけ学習させる。
    """
    認知ベクトル = 学習器.予測する("。").tolist()[0]
    最不利ラベル = 認知ベクトル.index(min(認知ベクトル))
    print("認知ベクトル:", 認知ベクトル, "\n最不利ラベル:", 最不利ラベル, "\n")
    for i in range(回数):
        文 = random.choice(データ[最不利ラベル])
        学習器.学習する(文, 最不利ラベル)
for i in range(1000):
    学習させる(学習器, [賛成派の文のリスト, 反対派の文のリスト], 5) # この行を編集

# 4.結果

学習が終わると、`学習器`は与えられた文からラベル番号を予測することができるようになる。  
これを利用して、偏向報道を定量的に評価しようというのだ。  

例えば、直前のセルの最後の2行が

```python
for i in range(1000):
    学習させる(学習器, [賛成派の文のリスト, 反対派の文のリスト], 5)
```

であった場合、  
「賛成派」の各文はベクトル$(1　0)$つまり$(\cos0^\circ　\sin0^\circ)$として学習されているし、  
「反対派」の各文はベクトル$(0　1)$つまり$(\cos90^\circ　\sin90^\circ)$として学習されている。  

今、マスコミ報道の文章を学習器に予測させ、出力されたベクトルが$(\cos80^\circ　\sin80^\circ)$だったとしよう。  
この場合、マスコミの報道は反対派に有利なように大きく偏っていることになる。  
$(\cos40^\circ　\sin40^\circ)$だった場合は、賛成派に有利なように少しだけ偏っている。  
真に中立公正な報道であればあるほど、$(\cos45^\circ　\sin45^\circ)$に近づくであろう。


In [ ]:
報道文 = # ここに、偏りを検出したい報道文を入力
print(学習器.予測する(報道文))